In [1]:
import pyspark
from pyspark.sql.functions import col,when,size,min, max,length
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master('local[*]')\
    .appName('topic_modelling_oth_langs')\
    .getOrCreate()

In [3]:
df_parquet = spark.read.parquet("part-00000-a8a717e8-bf83-42e3-a0b7-084971053d99-c000.snappy.parquet")

In [4]:
# create a column and put in the detected language code
from langdetect import detect
getCode = lambda x:detect(x)
sent = "this is eng"
getCode(sent)

'en'

In [5]:
# create a column and put in the detected language code
from langdetect import detect
# app_name_result = df_parquet.select(col('description')).collect()
# # create udf function to add a new column
# import pyspark.sql.functions as F
# from pyspark.sql.types import *

# def detect_lang(value):
#   return detect(value)

# #convert to a UDF Function by passing in the function and return type of function
# udf_detect_lang = F.udf(detect_lang, StringType())
# df_with_code = df_parquet.withColumn('lang_code', udf_detect_lang(df_parquet.description))
# df_with_code.show(5)

# easier method to convert to pandas then apply then function
df_parquet_pd = df_parquet.toPandas()
df_parquet_pd['lang_code'] = df_parquet_pd['description'].apply(getCode)
df_parquet_pd

,app_name,bundle,description,genres,os_platform,lang_code
0,Black Box - Movie Listing,958957112,Black Box Movie Listing App is a collection li...,"[Entertainment, Magazines & Newspapers]",IOS,en
1,Knife Throwing Max,1491396285,you will have fun in this game\n100+ challengi...,"[Games_Sports, Games_Role Playing]",IOS,en
2,爆料公社,1387765782,全台最大社群「爆料公社」APP 上線啦，我們把社群解放了，不只有勁爆的爆料內容，還貼心的加上...,[Social Networking],IOS,zh-tw
3,Lidow - Photo Editor & Collage,894532288,layout/grid/collage、Square/no crop for instagr...,[Photo & Video],IOS,en
4,Vidstitch Frames for Instagram,712908978,"■ #Vidstitch\n■ Featured on 148apps, Stelapps,...","[Photo & Video, Social Networking]",IOS,en
...,...,...,...,...,...,...
860,Barber Hair Salon & Beard Makeover,com.hmg.haircutgames.beardsalon.barberhaircut,Barber hair makeover salon game is for those w...,"[ENTERTAINMENT, FAMILY_PRETEND]",ANDROID,en
861,DJ Remix Offline 2020,com.mpro.djremixoffline2020,Dj Remix Offline 2020 Mp3 merupakan aplikasi m...,[MUSIC_AND_AUDIO],ANDROID,id
862,Sahih Bukhari (English),com.bangladroid.sahihbukhari,We have made this application from the book of...,[BOOKS_AND_REFERENCE],ANDROID,en
863,Two Minute English,com.astrobix.twominuteenglish,Improve your spoken English rapidly with the h...,[EDUCATION],ANDROID,en


## Export the code result to local for correction (if any)

In [6]:
# df_parquet_pd.to_excel("dataset_with_lang_code.xlsx",encoding = "utf_8_sig")
# input("input ENTER to continue:")
# df_parquet_pd = pd.read_excel("dataset_with_lang_code_correct.xlsx",encoding = "utf_8_sig")
# df_parquet_pd[df_parquet_pd['lang_code']=='ko']

In [7]:
df_parquet_pd.columns

Index(['app_name', 'bundle', 'description', 'genres', 'os_platform',
       'lang_code'],
      dtype='object')

In [8]:
count_pd = df_parquet_pd.groupby('lang_code').count()

In [9]:
count_pd.sort_values(by='app_name', ascending=False)

,app_name,bundle,description,genres,os_platform
lang_code,,,,,
en,686,686,686,686,686
id,82,82,82,82,82
vi,16,16,16,16,16
bn,15,15,15,15,15
ar,10,10,10,10,10
zh-cn,9,9,9,9,9
th,9,9,9,9,9
ko,9,9,9,9,9
ja,4,4,4,4,4


##  Map with language name

In [10]:
import pandas as pd
code_ref = pd.read_excel('lang_code.xlsx')

In [11]:
type(code_ref)

pandas.core.frame.DataFrame

In [12]:
code_ref

,lang_code,language_name
0,ab,Abkhazian
1,aa,Afar
2,af,Afrikaans
3,ak,Akan
4,sq,Albanian
...,...,...
180,xh,Xhosa
181,yi,Yiddish
182,yo,Yoruba
183,za,"Zhuang, Chuang"


In [13]:
df_parquet_pd = pd.merge(df_parquet_pd,code_ref,how = 'left',on = 'lang_code')

In [14]:
df_parquet_pd

,app_name,bundle,description,genres,os_platform,lang_code,language_name
0,Black Box - Movie Listing,958957112,Black Box Movie Listing App is a collection li...,"[Entertainment, Magazines & Newspapers]",IOS,en,English
1,Knife Throwing Max,1491396285,you will have fun in this game\n100+ challengi...,"[Games_Sports, Games_Role Playing]",IOS,en,English
2,爆料公社,1387765782,全台最大社群「爆料公社」APP 上線啦，我們把社群解放了，不只有勁爆的爆料內容，還貼心的加上...,[Social Networking],IOS,zh-tw,Chinese
3,Lidow - Photo Editor & Collage,894532288,layout/grid/collage、Square/no crop for instagr...,[Photo & Video],IOS,en,English
4,Vidstitch Frames for Instagram,712908978,"■ #Vidstitch\n■ Featured on 148apps, Stelapps,...","[Photo & Video, Social Networking]",IOS,en,English
...,...,...,...,...,...,...,...
860,Barber Hair Salon & Beard Makeover,com.hmg.haircutgames.beardsalon.barberhaircut,Barber hair makeover salon game is for those w...,"[ENTERTAINMENT, FAMILY_PRETEND]",ANDROID,en,English
861,DJ Remix Offline 2020,com.mpro.djremixoffline2020,Dj Remix Offline 2020 Mp3 merupakan aplikasi m...,[MUSIC_AND_AUDIO],ANDROID,id,Indonesian
862,Sahih Bukhari (English),com.bangladroid.sahihbukhari,We have made this application from the book of...,[BOOKS_AND_REFERENCE],ANDROID,en,English
863,Two Minute English,com.astrobix.twominuteenglish,Improve your spoken English rapidly with the h...,[EDUCATION],ANDROID,en,English


In [16]:
df = df_parquet_pd[df_parquet_pd.language_name.isnull()]

In [17]:
df_parquet_pd.groupby('language_name').count().sort_values(by='app_name', ascending=False)

,app_name,bundle,description,genres,os_platform,lang_code
language_name,,,,,,
English,686,686,686,686,686,686
Indonesian,82,82,82,82,82,82
Vietnamese,16,16,16,16,16,16
Bengali,15,15,15,15,15,15
Chinese,12,12,12,12,12,12
Arabic,10,10,10,10,10,10
Korean,9,9,9,9,9,9
Thai,9,9,9,9,9,9
French,4,4,4,4,4,4


In [18]:
df_spark = spark.createDataFrame(df_parquet_pd)

In [22]:
df_spark.select("app_name","lang_code","language_name").where(col("language_name")=='Korean').show()

+------------------------------+---------+-------------+
|                      app_name|lang_code|language_name|
+------------------------------+---------+-------------+
|                   더 스파이크|       ko|       Korean|
|                  土地面積換算|       ko|       Korean|
|    富豪麻將單機版（單機麻將）|       ko|       Korean|
|       逃脫遊戲 從日本祭典逃脫|       ko|       Korean|
|        대박맞고 : 무료 고스톱|       ko|       Korean|
|  무료 한판 고스톱 (무료 맞고)|       ko|       Korean|
| 멍멍 초성퀴즈 : 강아지 키우기|       ko|       Korean|
|             뉴스공장 팟캐스트|       ko|       Korean|
|민속 맞고 2 : 전통 고스톱 게임|       ko|       Korean|
+------------------------------+---------+-------------+



## Start building model with Chinese language

In [23]:
# load with only Chinese data
df_spark_cn = df_spark.filter(col("language_name")=='Chinese')

### #1: customize punctuation to chinese

In [24]:
# Step 1. Text cleasing with punctuations
# df_spark_cn = df_spark_cn.drop("description_clean")
from pyspark.sql.functions import regexp_replace
REGEX = '[「」【】；！_，、~：。？\-\\.!?@#$%^&*+/\d]' 
df_spark_cn = df_spark_cn.withColumn("description_clean",regexp_replace(df_spark.description,REGEX,' '))
df_spark_cn.show()

+-----------------------------------+--------------------+-------------------------------------+--------------------+-----------+---------+-------------+-------------------------------------+------------------------------------+
|                           app_name|              bundle|                          description|              genres|os_platform|lang_code|language_name|                    description_clean|                   description_token|
+-----------------------------------+--------------------+-------------------------------------+--------------------+-----------+---------+-------------+-------------------------------------+------------------------------------+
|                           爆料公社|          1387765782|    全台最大社群「爆料公社」APP 上...| [Social Networking]|        IOS|    zh-tw|      Chinese|      全台最大社群 爆料公社 APP 上...|    [全台最大社群「爆料公社」app,...|
|              DJ多多 - MC喊麦社会摇|          1039764887|  多多团队最新出品，中国最好用的dj...|[Entertainment, M...|        IOS|    zh-cn|      C

In [25]:
df_spark_cn.select("description","description_clean").show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------

In [26]:
df_spark_cn.count()

12

In [27]:
df_spark_cn.dtypes

[('app_name', 'string'),
 ('bundle', 'string'),
 ('description', 'string'),
 ('genres', 'array<string>'),
 ('os_platform', 'string'),
 ('lang_code', 'string'),
 ('language_name', 'string'),
 ('description_clean', 'string'),
 ('description_token', 'array<string>')]

### #2: get tokens in Chinese

In [28]:
# Step 2. Tokenization
df_spark_cn = df_spark_cn.drop("description_token")
import jieba
import pyspark.sql.functions as F
from pyspark.sql.types import *

def tokenF(str):
    return ",".join(jieba.cut(str))

udf_tokenF = F.udf(tokenF,StringType())

df_spark_cn = df_spark_cn.withColumn("description_token",udf_tokenF(df_spark_cn.description_clean))

In [30]:
df_spark_cn.dtypes

[('app_name', 'string'),
 ('bundle', 'string'),
 ('description', 'string'),
 ('genres', 'array<string>'),
 ('os_platform', 'string'),
 ('lang_code', 'string'),
 ('language_name', 'string'),
 ('description_clean', 'string'),
 ('description_token', 'string')]

In [31]:
df_spark_cn.select(col("description_token")).show()

+-------------------------------+
|              description_token|
+-------------------------------+
| 全,台,最大,社群, ,爆料,公社...|
| 多多,团队,最新,出品, ,中国,...|
|   句读, ,发现,文字,之美,
,
...|
|您,是否,因为,自己,的,发音,不...|
|可以,通過,喵,喵,影視,搜索,功...|
|  小說,閱讀, ,專業,的,小,說,...|
|恐龙,智力,拼图,精选,了,各种,...|
|篆书,查询,是,为,书法,及,篆刻...|
| 游戏,开始, ,小老鼠,从洞,中,...|
|    多彩,小,魔仙, , , ,认识,...|
|拥有,海量,热门,畅销小说, ,类...|
|    ■, ,输入,一个,字, ,搜索,...|
+-------------------------------+



### #3: prepare stopwords in Chinese

In [32]:
# Step 3. prepare stopword list
stopword_cn = pd.read_excel("stopword_cn.xlsx")

In [33]:
# prepare for stopwword list
splitF = lambda x: x.split(",")
stopword_cn['Stopwords_list'] = stopword_cn['Stopwords'].apply(splitF)

stopword_list = []
for list in stopword_cn['Stopwords_list']:
    stopword_list = stopword_list+list

In [34]:
stopword_cn.dtypes

No.                int64
Category          object
Stopwords         object
Stopwords_list    object
dtype: object

In [35]:
len(stopword_list)

721

### Build TF-IDF model

In [36]:
pd_cn = df_spark_cn.toPandas()

In [37]:
# Step 4. TF-IDF countvector, Transform documents to document-term matrix
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95,stop_words = stopword_list) #ignore terms that have a document frequency strictly higher than the given threshold
dtm = cv.fit_transform(pd_cn['description_token'])

C:\Users\ChengXiaotian\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['app', '不仅', '不光', '不单', '不只', '不外乎', '不如', '不妨', '不尽', '不尽然', '不得', '不怕', '不惟', '不成', '不拘', '不料', '不是', '不比', '不然', '不特', '不独', '不管', '不至于', '不若', '不论', '不过', '不问', '便于', '俺们', '别人', '别处', '别是', '别的', '别管', '别说', '巴巴', '并且', '彼时', '彼此', '按照', '本人', '本地', '本着', '本身', '比及', '比如', '比方', '罢了', '般的', '譬喻', '鄙人'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [38]:
dtm.shape

(12, 554)

In [39]:
feature_names = cv.get_feature_names()

In [40]:
feature_names

['agreement',
 'app',
 'apple',
 'com',
 'dj',
 'eduu',
 'go',
 'google',
 'html',
 'http',
 'https',
 'id',
 'ios',
 'itunes',
 'jd',
 'judouapp',
 'mc',
 'member',
 'policy',
 'privacy',
 'ready',
 'remix',
 'subscribe',
 'ui',
 'wifi',
 '一个',
 '一句',
 '一套',
 '一定',
 '一幅',
 '一款',
 '一群',
 '一起',
 '一边',
 '一键',
 '万部',
 '三五',
 '三五成群',
 '三要素',
 '上線',
 '下载',
 '不准',
 '不到',
 '不好',
 '不耗',
 '专属',
 '严肃',
 '並且',
 '中华',
 '中国',
 '中文',
 '中文舞曲',
 '中開',
 '丰富',
 '主要',
 '之破曉之戰',
 '之美',
 '九州',
 '书友',
 '书法',
 '书荒',
 '二指',
 '二次元',
 '互动',
 '互動',
 '交互',
 '交流',
 '交给',
 '产品',
 '亲子',
 '亲爱',
 '介紹',
 '从洞',
 '付款',
 '仙侠',
 '以下',
 '以前',
 '价格',
 '伊人',
 '会员',
 '伤感',
 '使用',
 '便于',
 '便捷',
 '俗女養',
 '信息',
 '修真',
 '俱全',
 '像是',
 '儿童',
 '入手',
 '內一樣',
 '內容',
 '全免费',
 '公众',
 '公社',
 '关注',
 '关闭',
 '内容',
 '内扣',
 '内置',
 '军事',
 '准确',
 '出品',
 '击中',
 '击打',
 '分数',
 '分類',
 '切换',
 '切換',
 '创造力',
 '判断',
 '到期',
 '刺激',
 '創屬',
 '功能',
 '加上',
 '加入',
 '加强',
 '动下',
 '励志',
 '動漫',
 '包含',
 '包年',
 '包月',
 '十万',
 '十二',
 '协议',
 '协调',
 '单词',
 '博物',
 '卡通',

In [41]:
cv.vocabulary_

{'最大': 303,
 '社群': 398,
 '爆料': 359,
 '公社': 95,
 'app': 1,
 '上線': 39,
 '我們': 242,
 '解放': 467,
 '內容': 92,
 '還貼': 521,
 '加上': 116,
 '遊戲化': 518,
 '功能': 115,
 '每個': 327,
 '互動': 64,
 '獲得': 366,
 '金幣': 530,
 '送禮物給': 506,
 '心儀': 222,
 '夜店': 170,
 '介紹': 71,
 '小圈圈': 194,
 '一群': 31,
 '有著': 308,
 '相同': 387,
 '興趣': 449,
 '朋友': 309,
 '一起': 32,
 '進入': 515,
 '討論': 470,
 '是非': 293,
 '旗下': 289,
 '社團': 397,
 '像是': 88,
 '爆廢': 358,
 '爆系': 360,
 '故事': 276,
 '美妝': 438,
 '將陸續': 191,
 '中開': 52,
 '而當': 440,
 '一定': 28,
 '創屬': 114,
 '這些': 512,
 '打賞給': 248,
 '對象': 193,
 '喜歡': 156,
 '文章': 283,
 '三五成群': 37,
 '進夜店': 516,
 '打屁': 247,
 '聊天': 441,
 '現實': 371,
 '內一樣': 91,
 'dj': 4,
 '播放歌曲': 270,
 '邀請': 519,
 '三五': 36,
 '好友': 176,
 '進來': 514,
 '狂歡': 365,
 '地圖': 162,
 '想要': 237,
 '知道': 394,
 '附近': 537,
 '美女': 437,
 '透過': 511,
 '立刻': 404,
 '顯示': 548,
 '身邊': 496,
 '並且': 47,
 '直接': 385,
 '還定': 520,
 '時開': 295,
 '啟寶箱': 153,
 '多多': 167,
 '团队': 157,
 '最新': 305,
 '出品': 103,
 '中国': 49,
 '最好': 304,
 '音乐盒': 547,
 '最赞': 306,
 '播放器': 

In [42]:
# Step 5. TfidfTransformer to compute the IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidfTransformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidfTransformer.fit(dtm)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [43]:
import numpy as np
def get_keyword_list(doc):
    # generate tfidf vector for given doc
    tf_idf_vector = tfidfTransformer.transform(cv.transform([doc]))
    
    # Return a C00rdinate representation of this matrix
    coo_matrix = tf_idf_vector.tocoo()
    # sorted_items = sort_coo(coo_matrix)

    # get Top-n index list
    index_sort = np.argsort(coo_matrix.data)[-topn:]
    
    keyword_list = []
    # extract keywords
    for idx in index_sort[::-1]: 
        keyword_list.append(feature_names[coo_matrix.col[idx]])
    
    return keyword_list

In [44]:
# Step 6. Get consolidated keyword list for entire dataset

# get topn = 10, keywords
topn = 10
pd_cn['description_keyword_list']=pd_cn['description_token'].apply(get_keyword_list)

In [45]:
pd_cn['description_keyword_list']

0            [爆料, 公社, 金幣, 小圈圈, 好友, app, 獲得, 地圖, 一起, 夜店]
1            [dj, 网络, 红歌, 喊麦, ios, 下载, 资源, 超劲, 播放器, 最好]
2     [句子, judouapp, 订阅, 续订, com, 周期, 连续, 自动, itunes...
3              [英语, 单词, 孩子, 发音, 小熊, 学习, 女声, 模仿, 句型, 同步]
4         [影視, 知否, 之破曉之戰, 火王, 热爱, 熱播劇, 男孩, 節目, 綜藝, 綠肥紅]
5              [閱讀, 熱門, 小說, 強大, 更新, 推薦, 功能, 經典, 產品, 盡有]
6      [恐龙, 智力, 拼图, 小朋友, ready, 进步, 精选, 玩耍, 爸爸妈妈, 拼图游戏]
7             [支持, 查询, 书法, 篆刻, 爱好者, 方便, 主要, 截图, 篆书, 简体]
8              [老鼠, 孩子, 能力, 培养, 游戏, 从洞, 整个, 锤子, 钻出, 过程]
9             [颜色, 宝宝, 填色, 右脑, 创造力, 认识, 画面, 培养, 开发, 游戏]
10             [小说, 热门, 灵异, 宫斗, 一款, 拥有, 拨动, 搞笑, 校园, 火爆]
11     [google, 词典, 字典, 汉语, eduu, 输入, 词霸, 范文, 网页, 维基百科]
Name: description_keyword_list, dtype: object

In [46]:
spark_cn = spark.createDataFrame(pd_cn)

In [47]:
len(pd_cn)

12

In [48]:
# create new columns to store keyword
for i in range(topn):
        spark_cn = spark_cn.withColumn("keyword-"+str(i+1),spark_cn['description_keyword_list'][i])

In [49]:
spark_cn.columns

['app_name',
 'bundle',
 'description',
 'genres',
 'os_platform',
 'lang_code',
 'language_name',
 'description_clean',
 'description_token',
 'description_keyword_list',
 'keyword-1',
 'keyword-2',
 'keyword-3',
 'keyword-4',
 'keyword-5',
 'keyword-6',
 'keyword-7',
 'keyword-8',
 'keyword-9',
 'keyword-10']

In [50]:
# save the final keyword spreadsheet to local
df_desc_final = spark_cn.toPandas()
df_desc_final.to_csv('desc_keyword_cn.csv',encoding='utf_8_sig')
# spark_cn.write.csv('desc_keyword_cn.csv')